## Cats and Dogs classifier using tf.data


In [0]:
#"""
# Google Collab specific stuff....
from google.colab import drive
drive.mount('/content/drive')

import os
!ls "/content/drive/My Drive"

USING_COLLAB = True
%tensorflow_version 2.x
#"""

In [0]:
# Setup sys.path to find MachineLearning lib directory

try: USING_COLLAB
except NameError: USING_COLLAB = False

%load_ext autoreload
%autoreload 2

import sys
if "MachineLearning" in sys.path[0]:
    pass
else:
    print(sys.path)
    if USING_COLLAB:
        sys.path.insert(0, '/content/drive/My Drive/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    else:
        sys.path.insert(0, '/Users/john/Documents/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
    print(sys.path)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, warnings, time, copy, csv, gc
import numpy as np 

import IPython.display as display
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from tqdm import tqdm_notebook, tnrange, tqdm
import pandas as pd

import tensorflow as tf
print(tf.__version__)

import tensorflow_datasets as tfds

AUTOTUNE = tf.data.experimental.AUTOTUNE
print("AUTOTUNE: ", AUTOTUNE)

from TrainingUtils import *

#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

## Examine and understand data


In [0]:
# GLOBALS/CONFIG ITEMS

# Set root directory path to data
if USING_COLLAB:
    ROOT_PATH = "/content/drive/My Drive/ImageData/DogsCats"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
else:
    ROOT_PATH = "/Users/john/Documents/ImageData/DogsCats"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
        
# Establish global dictionary
parms = GlobalParms(MODEL_NAME="model-CatsDogs-V01.h5",
                    ROOT_PATH=ROOT_PATH,
                    TRAIN_DIR="train", 
                    SMALL_RUN=False,
                    NUM_CLASSES=2,
                    IMAGE_ROWS=224,
                    IMAGE_COLS=224,
                    IMAGE_CHANNELS=3,
                    BATCH_SIZE=8,
                    EPOCS=10,
                    IMAGE_EXT=".jpg",
                    FINAL_ACTIVATION='sigmoid',
                    LOSS='binary_crossentropy',
                    METRICS=['accuracy'])

parms.print_contents()

In [0]:
# Simple helper method to display batches of images with labels....        
def show_batch(image_batch, label_batch, number_to_show=25, r=5, c=5, print_shape=False):
    show_number = min(number_to_show, parms.BATCH_SIZE)

    if show_number < 8: #if small number, then change row, col and figure size
        if parms.IMAGE_COLS > 64 or parms.IMAGE_ROWS > 64:
            plt.figure(figsize=(25,25)) 
        else:
            plt.figure(figsize=(10,10))  
        r = 4
        c = 2 
    else:
        plt.figure(figsize=(10,10))  

    if show_number == 1:
        image_batch = np.expand_dims(image_batch, axis=0)
        label_batch = np.expand_dims(label_batch, axis=0)

    for n in range(show_number):
        if print_shape:
            print("Image shape: {}  Max: {}  Min: {}".format(image_batch[n].shape, 
                                                             np.max(image_batch[n]), 
                                                             np.min(image_batch[n])))
        ax = plt.subplot(r,c,n+1)
        ax = plt.subplot(r,c,n+1)
        cmap="gray"
        if len(image_batch[n].shape) == 3:
            if image_batch[n].shape[2] == 3:
                cmap="viridis"
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_batch[n]), cmap=plt.get_cmap(cmap))

        plt.title(parms.CLASS_NAMES[np.argmax(label_batch[n])])

        plt.axis('off')


In [0]:
# Download dataset to local VM
full_dataset, info = tfds.load(name="cats_vs_dogs", with_info=True, split="train")
assert isinstance(full_dataset, tf.data.Dataset)
print(info)

In [0]:
# Set Class names...
parms.set_class_names(["Cat", "Dog"])
print("Classes: ", parms.NUM_CLASSES, 
      "   Labels: ", len(parms.CLASS_NAMES), 
      "  ", parms.CLASS_NAMES)


In [0]:
!ls "/root/tensorflow_datasets/cats_vs_dogs/4.0.0"

## Build an input pipeline

In [0]:
def image_aug(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image

def image_rescale_1_neg_1(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.resize(image, (parms.IMAGE_ROWS, parms.IMAGE_COLS))
    image = image / 255
    return image

def process_train(cat_dog_dict: tf.Tensor) -> tf.Tensor:
    image = cat_dog_dict["image"]
    label = cat_dog_dict["label"]
    image = image_rescale_1_neg_1(image)
    image = image_aug(image)
    return image, label_to_onehot(label)

def process_val(cat_dog_dict: tf.Tensor) -> tf.Tensor:
    image = cat_dog_dict["image"]
    label = cat_dog_dict["label"]
    image = image_rescale_1_neg_1(image)
    return image, label_to_onehot(label)

def label_to_onehot(label: tf.Tensor) -> tf.Tensor:
    return tf.one_hot(label, parms.NUM_CLASSES)

In [0]:
# split into training and validation sets of images
images_list_len = info.splits['train'].num_examples

train_len = int(0.9 * images_list_len)
val_len = images_list_len - train_len

# Create datasets with new sizes
train_dataset = full_dataset.take(train_len)  #  Creates dataset with new size
val_dataset = full_dataset.skip(train_len)  # Creates dataset after skipping over the size

steps_per_epoch = np.ceil(train_len // parms.BATCH_SIZE) # set step sizes based on train & batch
validation_steps = np.ceil(val_len // parms.BATCH_SIZE) # set step sizes based on val & batch

print("Total number: ", images_list_len, "  Train number: ", train_len, "  Val number: ", val_len)
print("Steps/EPOC: ", steps_per_epoch, "  Steps/Validation: ", validation_steps)

### Training setup

In [0]:
def cache_dataset(dataset, cache=False):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    return dataset

In [0]:
# Verify image paths were loaded and save one path for later in "some_image"
for f in train_dataset.take(2):
    some_image = f["image"]
    some_label = f["label"]
    print(some_label)

# map training images to pre-cache processing, includes any augmentation
train_dataset = train_dataset.map(process_train, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in train_dataset.take(1):
    print("Map 1 Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Label: ", np.argmax(label.numpy()), label.numpy())

# cache
train_dataset = cache_dataset(train_dataset, cache="./CatDogtrain1.tfcache")
#train_dataset = cache_dataset(train_dataset, cache=True)

# Repeat forever
train_dataset = train_dataset.repeat()

# set the batch size
train_dataset = train_dataset.batch(parms.BATCH_SIZE)


In [0]:
# Show the images, execute this cell multiple times to see the images

image_batch, label_batch = next(iter(train_dataset))
#show_batch(image_batch.numpy(), label_batch.numpy())
show_batch(image_batch.numpy(), label_batch.numpy(), print_shape=False, number_to_show=6)

### Validation setup

In [0]:
# Verify image paths were loaded and save one path for later in "some_image"
for f in val_dataset.take(2):
    some_label = f["label"]
    print(some_label)

# map training images to processing, includes any augmentation
val_dataset = val_dataset.map(process_val, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in val_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Label: ", np.argmax(label.numpy()), label.numpy())

# cache
val_dataset = cache_dataset(val_dataset, cache="./CatDogVal2.tfcache")
#val_dataset = cache_dataset(val_dataset, cache=True)

# Repeat forever
val_dataset = val_dataset.repeat()

# set the batch size
val_dataset = val_dataset.batch(parms.BATCH_SIZE)


In [0]:
# Test Validation, use smaller "number_to_show" to help show the augmentation

image_batch, label_batch = next(iter(val_dataset))
#show_batch(image_batch.numpy(), label_batch.numpy())
show_batch(image_batch.numpy(), label_batch.numpy(), number_to_show=6)

## Build  model
- add and validate pretrained model as a baseline

In [0]:
# Create any call backs for training...These are the most common.

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, min_lr=1e-6)
earlystopper = EarlyStopping(patience=8, verbose=1)
checkpointer = ModelCheckpoint(parms.MODEL_PATH, monitor='val_loss', verbose=1, mode="auto", save_best_only=True)
#csv_logger = CSVLogger(self.cvslogfile, append=True, separator=';')

#from keras.callbacks import TensorBoard
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))


In [0]:
# Create model and compile it

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D, Conv2DTranspose, Concatenate, Activation
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, Adam, Nadam, SGD
########

# https://www.tensorflow.org/api_docs/python/tf/keras/applications
from tensorflow.keras.applications import MobileNet, imagenet_utils, ResNet50
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

def predict_image(model_passed, image):  
    image = image_rescale_1_neg_1(image)   
    image = np.expand_dims(image, axis=0)
    predictions = model_passed.predict(image)
    return predictions 


def build_model(parms):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=parms.IMAGE_DIM))
    model.add(MaxPooling2D(pool_size=(5, 5), padding='same'))

    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), padding='same'))

    model.add(Conv2D(32, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), padding='same'))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))

    model.add(Dropout(0.25))

    model.add(Dense(parms.NUM_CLASSES, activation=parms.FINAL_ACTIVATION)) 
    return model

def compile_model(parms, model):
    # Optimizers: https://ruder.io/optimizing-gradient-descent/index.html#gradientdescentoptimizationalgorithms
    model.compile(loss=parms.LOSS,
          optimizer=SGD(lr=0.001, momentum=0.9),
          #optimizer="adam",
          #optimizer="rmsprop",
          metrics=parms.METRICS)
    return model


## Train model

In [0]:
# Train model
model = build_model(parms)
model = compile_model(parms, model)

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=parms.EPOCS, 
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[reduce_lr, earlystopper, checkpointer] # include any callbacks...
                    )

In [0]:
# Plot the training history
history_df = pd.DataFrame(history.history)
plt.figure()
history_df[['loss', 'val_loss']].plot(title="Loss")
plt.xlabel('Epocs')
plt.ylabel('Loss')
history_df[['accuracy', 'val_accuracy']].plot(title="Accuracy")
plt.xlabel('Epocs')
plt.ylabel('Accuracy')
plt.show()

## Validate model's predictions
- Create actual_lables and predict_labels
- Calculate Confusion Matrix & Accuracy
- Display results


In [0]:
#Load saved model
from tensorflow.keras.models import load_model 
def load_saved_model(model_path):
    model = load_model(model_path)
    print("loaded: ", model_path)
    return model

model = load_saved_model(parms.MODEL_PATH)

In [0]:
# Use model to generate predicted labels and probabilities
labels, predict_labels, predict_probabilities, bad_results = predictions_using_dataset(model, val_dataset, validation_steps, parms.BATCH_SIZE, create_bad_results_list=False)
#labels, predict_labels, predict_probabilities, bad_results = predictions_using_dataset(model, val_dataset, 1, parms.BATCH_SIZE, create_bad_results_list=False)


In [0]:
show_confusion_matrix(labels, predict_labels, parms.CLASS_NAMES)

In [0]:
# Graph the results
display_prediction_results(labels, predict_labels, predict_probabilities, parms.NUM_CLASSES, parms.CLASS_NAMES)


In [0]:
#Create a df from the bad results list, can save as csv or use for further analysis
bad_results_df = pd.DataFrame(bad_results, columns =['actual', 'predict', 'prob', 'image'])
bad_results_df.head()

In [0]:
# default is to not return bad_results, change to include them, create_bad_results_list=True

#bad_act, bad_pred, bad_prob, bad_images = zip(*bad_results)


In [0]:
# display images....        
def show_bad_batch(image_batch, bad_act, bad_pred, number_to_show=25):
    plt.figure(figsize=(10,10))
    show_number = number_to_show
    if len(image_batch) < number_to_show:
        show_number = len(image_batch)
        
    for n in range(show_number):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_batch[n]))
        #s = parms.CLASS_NAMES[bad_pred[n][0]]
        s = "Act: "+ str(bad_act[n][0]) + " Pred: " + str(bad_pred[n])
        plt.title(s)
        plt.axis('off')

In [0]:

#show_bad_batch(bad_images, bad_act, bad_pred)